In [28]:
# train 結果のlogをjson形式で保存
import json

class TrainLogging:
    
    def __init__(self):
        self.log = []
    
    def emit(self, **kwargs):
        self.log.append(kwargs)
    
    def save(self, path):
        with open(path, "w") as f:
            json.dump(self.log, f, indent=4)

In [29]:
# after
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim

def process(trainloader, testloader, model, epochs, lr, lr_scheduling=None, log_savepath=None):
    
    log_dict = defaultdict(list)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    if lr_scheduling is not None:
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_scheduling)
    
    def train(trainloader):
        sum_loss, sum_correct, sum_dataN = 0.0, 0, 0
        for (inputs, labels) in trainloader:
            optimizer.zero_grad()
            outputs, _ = model(inputs)
            loss = criterion(outputs, labels)
            sum_loss += loss.item()
            _, predicted = outputs.max(1)
            sum_dataN += labels.size(0)
            sum_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()
        train_loss = sum_loss*trainloader.batch_size/len(trainloader.dataset)
        train_acc = float(sum_correct/sum_dataN)
        return train_loss, train_acc
    
    def test(testloader):
        sum_loss, sum_correct, sum_dataN = 0.0, 0, 0
        for (inputs, labels) in testloader:
            outputs, _ = model(inputs)
            loss = criterion(outputs, labels)
            sum_loss += loss.item()
            _, predicted = outputs.max(1)
            sum_dataN += labels.size(0)
            sum_correct += (predicted == labels).sum().item()
        test_loss = sum_loss*testloader.batch_size/len(testloader.dataset)
        test_acc = float(sum_correct/sum_dataN)
        return test_loss, test_acc
    
    print("\n{0:<13}{1:<13}{2:<13}{3:<13}{4:<13}{5:<6}".format("epoch","train/loss","train/acc","test/loss","test/acc","lr"))
    logging = TrainLogging()
    for epoch in range(1, epochs + 1):
        train_loss, train_acc = train(trainloader)
        test_loss, test_acc = test(testloader)
        lr = optimizer.param_groups[-1]["lr"]
        print("{0:<13}{1:<13.5f}{2:<13.5f}{3:<13.5f}{4:<13.5f}{5:<6.6f}".format(epoch, train_loss, train_acc, test_loss, test_acc, lr))
        logging.emit(epoch=epoch, train_loss=train_loss, train_acc=train_acc, test_loss=test_loss, test_acc=test_acc, lr=lr)
        if lr_scheduling is not None: scheduler.step()
    if log_savepath is not None:
        logging.save(log_savepath)
    
    return model

In [2]:
# before
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim

def training(train, test, net, max_epoch, batch_size, initial_lr, lr_scheduling=None, logging=None):
    record = defaultdict(list)
    trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=initial_lr, momentum=0.9)
    if lr_scheduling is not None:
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_scheduling)
    print("\n{0:<13}{1:<13}{2:<13}{3:<13}{4:<13}{5:<6}".format("epoch","train/loss","train/acc","test/loss","test/acc","lr"))

    for epoch in range(max_epoch):
        # 学習モデルの訓練
        sum_loss = 0.0  # lossの合計
        sum_correct = 0 # 正解数の合計
        sum_data_num = 0  # データ数の合計
        for (inputs, labels) in trainloader:
            optimizer.zero_grad()
            outputs, _ = net(inputs)
            loss = criterion(outputs, labels)
            sum_loss += loss.item()
            _, predicted = outputs.max(1)
            sum_data_num += labels.size(0)
            sum_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()
        if lr_scheduling is not None:
            scheduler.step()
        lr = optimizer.param_groups[-1]["lr"]
        train_loss = sum_loss*batch_size/len(trainloader.dataset)
        train_acc = float(sum_correct/sum_data_num)
        record["train_loss"].append(train_loss)
        record["train_acc"].append(train_acc)
        # 学習モデルのテスト
        sum_loss = 0.0
        sum_correct = 0
        sum_data_num = 0
        for (inputs, labels) in testloader:
            outputs, _ = net(inputs)
            loss = criterion(outputs, labels)
            sum_loss += loss.item()
            _, predicted = outputs.max(1)
            sum_data_num += labels.size(0)
            sum_correct += (predicted == labels).sum().item()
        test_loss = sum_loss*batch_size/len(testloader.dataset)
        test_acc = float(sum_correct/sum_data_num)
        record["test_loss"].append(test_loss)
        record["test_acc"].append(test_acc)
        print("{0:<13}{1:<13.5f}{2:<13.5f}{3:<13.5f}{4:<13.5f}{5:<6.6f}".format(epoch+1, train_loss, train_acc, test_loss, test_acc, lr))
        if logging is not None:
            logging.emit(log={
                "epoch": epoch+1,
                "train/loss": train_loss,
                "train/acc": train_acc,
                "test/loss": test_loss,
                "test/acc": test_acc,
                "lr": lr
            })

    return net, record

In [3]:
# net のstub
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self, out=3):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, padding=1) # (1) 32*32*3 -> 32*32*16
        self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1) # (3) 16*16*16 -> 16*16*32
        self.conv3 = nn.Conv2d(32, 64, 3, 1, padding=1) # (5) 8*8*32 -> 8*8*64
        self.fc1 = nn.Linear(4*4*64, 500)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500, out)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2) # (2) 32*32*16 -> 16*16*16
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2) # (4) 16*16*32 -> 8*8*32
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2) # (6) 8*8*64 -> 4*4*64
        x = x.view(-1, 4*4*64)
        x = F.relu(self.fc1(x))
        feature = x
        x = self.dropout1(x)
        x = self.fc2(x)
        return x, feature

In [4]:
# trainloader, testloader を生成するstub
import torchvision
import torchvision.transforms as transforms

def update_labels(train, test):
    updated = [[], []]
    mapping_dict = defaultdict(lambda: -1)
    for i, t in enumerate([train, test]):
        t = sorted(t, key=lambda x:x[1])
        new_label = 0
        for data in t:
            if mapping_dict[data[1]] == -1:
                mapping_dict[data[1]] = new_label
                new_label += 1
            updated[i].append((data[0], mapping_dict[data[1]]))
    train, test = updated
    return train, test

def dataset_stub():
    path = "../../prototype/data/"
    classes = [1,2,8]
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    train = torchvision.datasets.CIFAR10(root=path, train=True, download=True, transform=transform)
    test = torchvision.datasets.CIFAR10(root=path, train=False, download=True, transform=transform)
    train = [d for d in train if d[1] in classes]
    test = [d for d in test if d[1] in classes]
    train, test = update_labels(train, test)
    return train, test

def loader_stub():
    batch_size = 128
    train, test = dataset_stub()
    trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)
    return trainloader, testloader

In [31]:
# lr_scheduling のstub
def lr_v1(epoch):
    if epoch < 10:
        return 1
    elif epoch < 20:
        return 0.1**1
    elif epoch < 30:
        return 0.1**2
    else:
        return 0.1**3

In [84]:
# before のtest
model = LeNet(3)
train, test = dataset_stub()
net, record = training(train, test, model, max_epoch=10, batch_size=128, initial_lr=0.001)

Files already downloaded and verified
Files already downloaded and verified

epoch        train/loss   train/acc    test/loss    test/acc     lr    
1            1.10220      0.33600      1.11694      0.33833      0.001000
2            1.09006      0.35200      1.10057      0.36133      0.001000
3            1.06595      0.42940      1.06700      0.50100      0.001000
4            1.01732      0.55700      0.99853      0.61100      0.001000
5            0.94776      0.60353      0.92929      0.61167      0.001000
6            0.88837      0.63113      0.87150      0.65667      0.001000
7            0.82806      0.66220      0.80250      0.68400      0.001000
8            0.75317      0.69973      0.72058      0.72133      0.001000
9            0.67574      0.73327      0.65743      0.73833      0.001000
10           0.62952      0.75413      0.62323      0.75433      0.001000


In [32]:
# after のtest (lr_scheduling, log_savepath あり)
model = LeNet(3)
trainloader, testloader = loader_stub()
model = process(trainloader, testloader, model, epochs=15, lr=0.001, lr_scheduling=lr_v1, log_savepath="./assets/log.json")

Files already downloaded and verified
Files already downloaded and verified

epoch        train/loss   train/acc    test/loss    test/acc     lr    
1            1.10389      0.35540      1.12028      0.36367      0.001000
2            1.09740      0.36967      1.11231      0.38333      0.001000
3            1.08748      0.39453      1.09845      0.43500      0.001000
4            1.06639      0.45820      1.06693      0.49833      0.001000
5            1.02018      0.53713      1.00368      0.56167      0.001000
6            0.95287      0.57907      0.92906      0.60500      0.001000
7            0.88587      0.62253      0.86304      0.65100      0.001000
8            0.81300      0.66540      0.79114      0.68033      0.001000
9            0.74188      0.69760      0.72242      0.70700      0.001000
10           0.68208      0.72280      0.66775      0.73867      0.001000
11           0.64819      0.74367      0.65927      0.74467      0.000100
12           0.64417      0.74393    